#### Days Chosen: 
8th January 2014 <br>
8th July 2014

#### Tags Chosen:

1. TEMP: HOT/COLD
2. LOCATION: ALBERMARLE/CHARLOTTESVILLE
3. TOTSQFT: SMALL/MEDIUM/LARGE
4. RMSP (Number of Rooms): 1-4, 5-8, 9+

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import seaborn as sns

In [ ]:
# df_cv_cold = pd.read_csv(r'VA540-20140108.csv')
# df_cv_cold['temp'] = "cold"
# df_cv_cold['loc'] = "cv"

df_cv_hot = pd.read_csv(r'VA540-20140708.csv')
df_cv_hot['temp'] = "hot"
df_cv_hot['loc'] = "cv"

# df_al_cold = pd.read_csv(r'VA003-20140108.csv')
# df_al_cold['temp'] = "cold"
# df_al_cold['loc'] = "al"

df_al_hot = pd.read_csv(r'VA003-20140708.csv')
df_al_hot['temp'] = "hot"
df_al_hot['loc'] = "al"

In [ ]:
df_cv_cold.shape

In [ ]:
df_al_cold.shape

In [ ]:
df_al_hot.shape

In [ ]:
df_cv_hot.shape

In [ ]:
# combined_df = pd.concat([df_cv_cold, df_cv_hot, df_al_cold, df_al_hot], ignore_index=True)

In [ ]:
combined_df = pd.concat([df_cv_hot, df_al_hot], ignore_index=True)

In [ ]:
combined_df.columns

In [ ]:
columns_to_keep = [f'total_kwh_{i}' for i in range(1, 25)]
# columns_to_keep.append('temp')
columns_to_keep.append('loc')
columns_to_keep.append('hid')

In [ ]:
df = combined_df.drop(columns=[col for col in combined_df.columns if col not in columns_to_keep])
df.head()

In [ ]:
df.shape

In [ ]:
row_max_values = df.iloc[:, 1:-1].max(axis=1) # Find the maximum value for each row (axis=1)

normalized_df = df.iloc[:, 1:-1].div(row_max_values, axis=0) # Divide each value by its corresponding max value

In [ ]:
normalized_df.head()

In [ ]:
hourly_usage = normalized_df.iloc[:, 0:24].values

In [ ]:
# Apply K-Means clustering

wcss = []

for n_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(hourly_usage)
    wcss.append(kmeans.inertia_)

In [ ]:
# Plot the WCSS values against the number of clusters

plt.figure(figsize=(10, 6))
plt.plot(range(2, 11), wcss, marker='o')
plt.title('Knee Method for Optimal Cluster Number')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS (Within-Cluster Sum of Squares)')
plt.xticks(range(0, 11))
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(hourly_usage)

In [ ]:
df['cluster'] = cluster_labels

# Print the number of households in each cluster
print(df['cluster'].value_counts())

In [ ]:
kmeans.fit(hourly_usage)
cluster_centers = kmeans.cluster_centers_

In [ ]:
# Create a time array for the x-axis (24 hours)
hours = np.arange(24)
plt.figure(figsize=(10, 6))  # Adjust the width and height as needed

# Plot each cluster's time series curve
for i, center in enumerate(cluster_centers):
    print(center.size)
    plt.plot(hours, center, label=f'Cluster {i}')
    
plt.xticks(hours)  
plt.xlabel('Hour')
plt.ylabel('Value')
plt.title('Cluster Center')
plt.legend()
plt.show()

In [ ]:
df['cluster'].unique()

In [ ]:
df_recs = pd.read_csv("tag_data/recs2015_public_v2.csv")
df_dem = pd.read_csv("tag_data/va_hh_51003_51540.csv")

In [ ]:
df

In [ ]:
# NUMBER OF ALB HOUSEHOLDS IN DEM DATA 
prefix_count = (df['hid'].astype(str).str.startswith('51003')).sum()
prefix_count

In [ ]:
prefix_count = (df['hid'].astype(str).str.startswith('51540')).sum()
prefix_count

In [ ]:
df_dem.shape

In [ ]:
merged_df = pd.merge(df, df_dem, on='hid', how='inner')

In [ ]:
merged_df.shape

In [ ]:
merged_df.head()

In [ ]:
columns_to_keep = ['DOEID', 'TOTSQFT_EN', 'KOWNRENT']
df_recs = df_recs[columns_to_keep]

In [ ]:
merged_df = pd.merge(df_recs, merged_df, left_on='DOEID', right_on='rid', how='inner')

In [ ]:
merged_df.shape

In [ ]:
sqft_intervals = [400, 1500, 3500,10000]

merged_df['SQFT_TAG'] = pd.cut(merged_df['TOTSQFT_EN'], bins=sqft_intervals, labels=['small', 'medium', 'large'], right=False)
h_counts = merged_df['SQFT_TAG'].value_counts()


print(h_counts)


In [ ]:
rmsp_intervals = [1,5,9,20]  

merged_df['RMSP_TAG'] = pd.cut(merged_df['RMSP'], bins=rmsp_intervals, labels=['1-4', '5-8', '9+'], right=False)
unit_counts = merged_df['RMSP_TAG'].value_counts()


print(unit_counts)

In [ ]:
ybl_intervals = [1,4,6,8,float('inf')]  

merged_df['YBL_TAG'] = pd.cut(merged_df['YBL'], bins=ybl_intervals, labels=['1959 or earlier','1960-1979','1980-1999','2000-2013'], right=False)
unit_counts = merged_df['YBL_TAG'].value_counts()


print(unit_counts)

In [ ]:
hincp_intervals = [float('-inf'),40000,100000,160000,float('inf')]  

merged_df['HINCP_TAG'] = pd.cut(merged_df['HINCP'], bins=hincp_intervals, labels=['Low income', 'Middle income', 'Upper-middle income','High income'], right=False)
unit_counts = merged_df['HINCP_TAG'].value_counts()


print(unit_counts)

In [ ]:
# UNITS IN STRUCTURE

unit_intervals = [0,2,3,4,10,float('inf')]  

merged_df['BLD_TAG'] = pd.cut(merged_df['BLD'], bins=unit_intervals, labels=['Mobile Home/Trailer', 'One-family house A', 'One-family house D','Apts','Mobile Home/Trailer'], right=False, ordered=False)
unit_counts = merged_df['BLD_TAG'].value_counts()


print(unit_counts)

In [ ]:
# NUMBER OF PERSONS

person_intervals = [1,2,3, float('inf')]  

merged_df['NP_TAG'] = pd.cut(merged_df['NP'], bins=person_intervals, labels=['1','2', '2+'], right=False)
per_counts = merged_df['NP_TAG'].value_counts()


print(per_counts)

In [ ]:
merged_df.iloc[0]

In [ ]:
cols = ['temp', 'loc', 'SQFT_TAG','RMSP_TAG','YBL_TAG','HINCP_TAG','BLD_TAG','NP_TAG']

for col in cols:
    plt.figure(figsize=(12, 6))
    sns.countplot(data=merged_df, x=col, order=merged_df[col].value_counts().index)
    plt.title(f'Frequency Analysis of {col} (Whole Dataset)')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
for col in cols:
    plt.figure(figsize=(15, 8))
    sns.countplot(data=merged_df, x=col, hue='cluster', order=merged_df[col].value_counts().index)
    plt.title(f'Frequency Analysis of {col} by Cluster')
    plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Specify values you're looking for
val1 = 0
val2 = 0
val3 = 0
tag = "NP_TAG"
y_1 = '1'
y_2 = '2'
y_3 = '2+'


# Count the number of rows where ColumnA has x_value and ColumnB has y_value
count_matching_rows_1 = len(merged_df[(merged_df['cluster'] == val1) & (merged_df[tag] == y_1)])
count_matching_rows_2 = len(merged_df[(merged_df['cluster'] == val1) & (merged_df[tag] == y_2)])
count_matching_rows_3= len(merged_df[(merged_df['cluster'] == val1) & (merged_df[tag] == y_3)])

print(f"Number of rows where ColumnA is {val1} and ColumnB is {y_1}: {count_matching_rows_1}")
print(f"Number of rows where ColumnA is {val2} and ColumnB is {y_2}: {count_matching_rows_2}")
print(f"Number of rows where ColumnA is {val3} and ColumnB is {y_3}: {count_matching_rows_3}")

In [ ]:
def format_to_input_file(df, alpha, beta, additional_tags):
    
    n = len(df)
    K = df['cluster'].nunique()

    unique_tags = set(df[additional_tags].values.flatten()) # Extract unique values from specified tag columns
    N = len(unique_tags)
    print(N)
    print(unique_tags)

    with open('data_hot_day.txt', 'w') as file:
        file.write(f"{n} {K} {N} {alpha} {beta}\n")
        file.write(f"Index Cluster {' '.join(unique_tags)}\n")

        # Write the data items
        for index, row in df.iterrows():
            cluster_number = row['cluster']
            tag_set = [1 if str(tag) in row[additional_tags].values else 0 for tag in unique_tags]

            # Write the line for each data item
            file.write(f"{index + 1} {cluster_number} {' '.join(map(str, tag_set))}\n")

In [ ]:
additional_tags = ['loc', 'SQFT_TAG','RMSP_TAG','YBL_TAG','HINCP_TAG','BLD_TAG','NP_TAG']
format_to_input_file(merged_df, alpha=4, beta=0, additional_tags=additional_tags)